```
kernel void simple(global int *restrict s1,
                   global const int *s2,
                   int foo)
{
    const int id = get_local_id(0);
    s1[id] = s2[id] + id * foo;
}


```


```bash
  auto device = xrt::device(0);
  auto uuid = device.load_xclbin(xclbin_fnm);
  auto simple = xrt::kernel(device, uuid.get(), "simple");
  auto bo0 = xrt::bo(device, DATA_SIZE, XCL_BO_FLAGS_NONE, simple.group_id(0));
  auto bo1 = xrt::bo(device, DATA_SIZE, XCL_BO_FLAGS_NONE, simple.group_id(1));
  bo0.sync(XCL_BO_SYNC_BO_TO_DEVICE, DATA_SIZE, 0);
  bo1.sync(XCL_BO_SYNC_BO_TO_DEVICE, DATA_SIZE, 0);

  auto run = simple(bo0, bo1, 0x10);
  run.wait();

  bo0.sync(XCL_BO_SYNC_BO_FROM_DEVICE, DATA_SIZE, 0);
```


In [ ]:
!cp /group/xlabs-co/grahams/XRTschelleg/tests/xrt/02_simple/kernel.hw_emu.xclbin .

In [ ]:
import os
os.environ["EMCONFIG_PATH"] = "/opt/xrtbind/XRTschelleg/tests/xrt"
os.environ['XCL_EMULATION_MODE'] = 'hw_emu'

In [ ]:
import numpy as np
bo0_data = np.zeros(1024, np.dtype('>i4')) 
bo1_data = np.array([range(1024)], np.dtype('>i4'))

data_size = bo1_data.nbytes
data_size

In [ ]:
import pyxrt

pyxrt.xclProbe()
d = pyxrt.device(0)

xclbin = './kernel.hw_emu.xclbin'
uuid = d.load_xclbin(xclbin)

In [ ]:
simple = pyxrt.kernel(d, uuid.get(), "simple", False)

In [ ]:
bo0 = pyxrt.bo(d, data_size, pyxrt.XCL_BO_FLAGS_NONE, simple.group_id(0))
bo1 = pyxrt.bo(d, data_size, pyxrt.XCL_BO_FLAGS_NONE, simple.group_id(1))

In [ ]:
# Running all binds - kernel() hack to take exact args PASS!
from wurlitzer import sys_pipes

with sys_pipes():
    bo0.write(bo0_data,0)
    bo1.write(bo1_data,0)
    bo0.sync(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)
    bo1.sync(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)
    
    run = simple(bo0, bo1, 0x10)
    run.wait()

In [ ]:
bo0.sync(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_FROM_DEVICE, data_size, 0)
bo0.read(4096,0)

### scratch

In [ ]:
# Running all binds
from wurlitzer import sys_pipes

with sys_pipes():
    bo0.write2(bo0_data,0)
    bo1.write2(bo1_data,0)
    bo0.sync2(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)
    bo1.sync2(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)
    
    args = bo0, bo1, 0x10
    run = simple(*(bo0, bo1, 0x10))
    run.wait()
    
    #pyxrt.partial_app(d, uuid, simple, bo0, bo1)

In [ ]:
# Using the partial function
from wurlitzer import sys_pipes

with sys_pipes():
    bo0.write2(bo0_data,0)
    bo1.write2(bo1_data,0)
    bo0.sync2(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)
    bo1.sync2(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)
    
    #args = bo0, bo1, 0x10
    #run = simple(*args)
    # run.start()
    #run.wait()
    
    pyxrt.partial_app(d, uuid, simple, bo0, bo1)

In [ ]:
args = bo0, bo1, 0x10
type(*args)

In [ ]:
import pyxrt

pyxrt.xclProbe()
d = pyxrt.device(0)

xclbin = './kernel.hw_emu.xclbin'
uuid = d.load_xclbin(xclbin)

simple = pyxrt.kernel(d, uuid.get(), "simple", False)

bo0 = pyxrt.bo(d, data_size, pyxrt.XCL_BO_FLAGS_NONE, simple.group_id(0))
bo1 = pyxrt.bo(d, data_size, pyxrt.XCL_BO_FLAGS_NONE, simple.group_id(1))

In [ ]:
# from wurlitzer import sys_pipes
# 
# with sys_pipes():
#     bo0.write(bo0_data,0)
#     bo1.write(bo1_data,0)

In [ ]:
from wurlitzer import sys_pipes

with sys_pipes():
    bo0.write2(bo0_data,0)
    bo1.write2(bo1_data,0)

In [ ]:
with sys_pipes():
    bo0.sync2(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)
    bo1.sync2(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)

    run = simple(bo0, bo1, 0x10)
    run.wait()

    bo0.sync2(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_FROM_DEVICE, data_size, 0)

In [ ]:
bo0.read2(data_size, 0)

In [ ]:
pyxrt.add_arrays(bo0_data, bo1_data)

In [ ]:
a = np.array(0, 'f8')
type(a)

In [ ]:
bo0_readback = np.frombuffer(bo0.read2(data_size, 0), dtype=np.dtype('>i4'))
bo1_readback = np.frombuffer(bo1.read2(data_size, 0), dtype=np.dtype('>i4'))
print(list(bo0_readback[:10]))
print(list(bo1_readback[:10]))

In [ ]:
bo0.read2(data_size, 0)
bo1.read2(data_size, 0)

In [ ]:
x = bo0.read2(data_size, 0)
x

In [ ]:
# buffer object attempt
bo0_buf = np.array(bo0, copy=False)
bo0_buf.fill(1)
bo0_buf

In [ ]:
# buffer object attempt
bo1_buf = np.array(bo1, copy=False)
bo1_buf.fill(5)
bo1_buf

In [ ]:
bo2_buf = np.array(bo0, copy=False)
bo2_buf

In [ ]:
bo0_map = bo0.map(data_size)
bo1_map = bo1.map(data_size)

In [ ]:
bo0_map.fill(1)
bo1_map.fill(5)

In [ ]:
print(list(bo0_map.astype(np.dtype('>i4'))))
print(list(bo1_map.astype(np.dtype('>i4'))))

In [ ]:
bo0_map2 = bo0.map(data_size)
bo0_map3 = bo0.map(data_size)
bo0_map4 = bo0.map(data_size)
bo0_map5 = bo0.map(data_size)

In [ ]:

print(list(bo0_map2.astype(np.dtype('>i4')))[:10])
print(list(bo0_map3.astype(np.dtype('>i4')))[:10])
print(list(bo0_map4.astype(np.dtype('>i4')))[:10])
print(list(bo0_map5.astype(np.dtype('>i4')))[:10])

In [ ]:
def getpointer(a):
    pointer, read_only_flag = a.__array_interface__['data']
    return(pointer)

for p in [bo0_map2,bo0_map3,bo0_map4,bo0_map5]:
    print(f'{getpointer(p)} {p[0]}')

In [ ]:
bo0_map2[1] == bo0_map[1]

In [ ]:
print(list(bo0_map.astype(np.dtype('>i4')))[:10])
print(list(bo0_map2.astype(np.dtype('>i4')))[:10])
print(list(bo0_map3.astype(np.dtype('>i4')))[:10])

In [ ]:
bo0_map3

In [ ]:
bo0_map2

In [ ]:
bo0_map

In [ ]:
bo0.write(bo0_data,0)
bo1.write(bo1_data,0)

In [ ]:
bo0_readback = np.frombuffer(bo0.read(data_size, 0), dtype=np.dtype('>i4'))
bo1_readback = np.frombuffer(bo1.read(data_size, 0), dtype=np.dtype('>i4'))
print(list(bo0_readback[:10]))
print(list(bo1_readback[:10]))

In [ ]:
pointer, read_only_flag = bo0_map.__array_interface__['data']
pointer

In [ ]:
foo = 0x10
        
expected = [i+i*foo for i in range(1024)]
print(expected[:10])

In [ ]:
bo0_readback2 = np.frombuffer(bo0.read(data_size, 0), dtype=np.dtype('>i4'))
bo1_readback2 = np.frombuffer(bo1.read(data_size, 0), dtype=np.dtype('>i4'))
print(list(bo0_readback2[:10]))
print(list(bo1_readback2[:10]))

In [ ]:
bo0.sync(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_FROM_DEVICE, data_size, 0)
bo1.sync(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_FROM_DEVICE, data_size, 0)

print(list(bo0_map.astype(np.dtype('>i4'))))
print(list(bo1_map.astype(np.dtype('>i4'))))

In [ ]:
from wurlitzer import sys_pipes

with sys_pipes():
    bo0.sync(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)
    bo1.sync(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_TO_DEVICE, data_size, 0)


In [ ]:
from wurlitzer import sys_pipes

with sys_pipes():
    run = simple(bo0, bo1, 0x10)
    run.wait()

In [ ]:
with sys_pipes():
    bo0.sync(pyxrt.xclBOSyncDirection.XCL_BO_SYNC_BO_FROM_DEVICE, data_size, 0)

In [ ]:
bo0_readback = np.frombuffer(bo0.read(data_size, 0), dtype=np.dtype('>i4'))
list(bo0_readback)

In [ ]:
del bo0
del bo1
del simple
del d

In [ ]:
import numpy as np
print(np.dtype('>i4').itemsize)

In [ ]:
import os
import pyxrt
os.path.abspath(pyxrt.__file__)